In [0]:
# Importations
import sys      
import urllib
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup, Tag, NavigableString, Comment
import pickle
from collections import defaultdict
from google.colab import drive
import re

In [0]:
try:
  # ouvrir une page web
    html = urlopen('https://www.tui.fr/faq/reservation/#commentfaire')
except (HTTPError, URLError) as e:
    sys.exit(e)   # sortie du programme avec affichage de l’erreur

# Représente l'arbre html global
bsObj = BeautifulSoup(html, "lxml")

from collections import defaultdict
import regex as re

maDiv = bsObj.find("div", {"class":"max-container tabs-left"}).find_all('li')#.find_all('button').contents

On apporte ensuite des modifications pour les hyperliens.

In [0]:
dico_result=defaultdict(dict)
for elt in maDiv:
  for classes in elt.find_all('button'):
    questions = []
    reponses = []
    for quest in elt.find('div', {"class":"accordion-list"}).find_all('h3'):
      questions.append(re.sub("[ ]{2,}"," ",quest.get_text().replace('\r', '').replace('\n', ' ').replace("\\","").strip()))
    for rep in elt.find('div', {"class":"accordion-list"}).find_all('p'):
      prem = str(rep).replace('<br/>','').replace('<p>', '').replace('</p>', '').strip().replace('\n', ' ').replace('\r', '').replace("</strong>",'').replace("<strong>",'').replace('<b>','').replace('</b>','')
      regURL = r"<a.+?\s*href=\"(?P<url>.+?)\"\s*.+?\s*>" 
      result = re.findall(regURL,prem,re.I)
      balise = re.findall(r"<a.+?>",prem, re.I)
      if len(balise) != 0 :
        for res in result :
          if res.startswith("/") :
            prem = prem.replace("</a>"," https://www.tui.fr"+res , 1 )
          else : 
            prem = prem.replace("</a>", " "+res, 1)
        for bal in balise :
          prem = prem.replace(bal,"")  
      prem = re.sub("[ ]{2,}"," ",prem)
      reponses.append(prem)
      dico = dict(zip(questions, reponses))
    dico_result[re.sub("[ ]{2,}"," ",classes.get_text().replace('\r', '').replace('\n', ' ').replace("\\","").strip())] = dico

In [0]:
try:
  # ouvrir une page web
    html = urlopen('https://www.exotismes.fr/voyages/t/FAQ.html#restauration-01')
except (HTTPError, URLError) as e:
    sys.exit(e)   # sortie du programme avec affichage de l’erreur

# Représente l'arbre html global
bsObj = BeautifulSoup(html, "lxml")

maDiv = bsObj.find("div", id="j_id_t:0:spcorptext")

#On cherche le premier '\xa0' car on s'intéresse qu'à la partie après le premier <p> <\p>.
#On stocke cet index dans la variable idx
idx = 0
for i,e in enumerate(maDiv.find_all()):
  if e.string =='\xa0' :
    idx = i 
    break;
#On veut les éléments qu'après la 233 ième valeur

scrap = maDiv.find_all()[idx:]

dic_general = defaultdict(dict)

Schéma des données : 

Balise div avec la question 

Balises p et b avec la question 

Balise b avec la question 

Balise p avec la réponse 

Si url dans réponse : on a un a et un br 

p vide pour séparer les questions et plusieurs si séparation de partie

In [0]:
crt = '' #Partie courante
q_crt = '' #Question courante
prec_b = False 

for elt in scrap : 
  
  if elt.name == 'h2' and elt.get_text()!=crt : #Partie
    crt = elt.get_text() 
    dic_general[crt] = defaultdict(dict) #On crée un dico pour cette partie
    prec_b = False

  elif elt.name == 'div' and elt.find('b').contents[0]!=q_crt : #Question
    q_crt = elt.find('b').contents[0] #On change la question courante
    prec_b = False

  elif elt.name =="b" : 
    prec_b = True #Une réponse est précédé d'une balise b, on vérifie donc. 

  elif elt.name =="p" and prec_b == True and elt.get_text()!= '\xa0' : 
    temp = elt.get_text().replace('\r','').replace('\n',' ').replace('\xa0','') #nettoyage
    #remplacement des liens hypertexte par les liens :
    temp = temp.replace('Assurance Voyage MULTIRISQUES Exotismes', 'https://www.exotismes.fr/documents/assurances/')\
    .replace("Formalités d'entrée par destination", 'https://www.exotismes.fr/voyages/t/formalites-entree.html')\
    .replace("Détail Assurance", "https://www.exotismes.fr/documents/assurances/pdf/assurinco_3199.pdf")\
    .replace("Conditions Générales de Ventes", "https://www.exotismes.fr/documents/pdf/cond_vente.pdf")\
    .replace("Détails Assurances MULTIRISQUES", "https://www.exotismes.fr/documents/assurances/pdf/assurinco_3199.pdf")\
    .replace('Compagnies Aériennes', "https://www.exotismes.fr/voyages/t/compagnies-aeriennes.html")
    dic_general[crt][q_crt] = temp
     
    prec_b = False

  elif elt.name=='p' and elt.get_text()== '\xa0' and prec_b == True : 
    prec_b = True

  else : 
    prec_b = False

In [7]:
print(dico_result.keys())
print(dic_general.keys())

dict_keys(['Réservation en ligne', 'Tarifs', 'Vols', 'Circuits', 'Séjours', 'Après le retour', 'Informations voyageurs', 'À propos de TUI'])
dict_keys(['Assurance', 'Médical & Sanitaire', 'Rapatriement', 'Restauration', 'Service Client', 'Service Réservation', 'Paiement', 'Problème Informatique', 'Séjour/Voyage', 'Enfants & Bébés'])


Les classes Séjours et Séjours/Voyages des deux sites peuvent être regroupées.

In [0]:
faq = defaultdict(dict)
key1 = list(dico_result.keys())
key1.remove('Séjours')
key2 = list(dic_general.keys())

for classe in key1 : 
  faq[classe] = dico_result[classe]
for classe in key2:
  if classe=='Séjours/Voyages' : 
    faq[classe] = dict(dico_result['Séjours'].items() + dic_general[classe].items())
  else : 
    faq[classe] = dic_general[classe]

On peut ensuite enregistrer notre faq dans un fichier pickle.

In [9]:
faq

defaultdict(dict,
            {'Après le retour': {'Comment puis-je percevoir le remboursement des taxes aériennes ?': "Lorsque vous ne prenez pas votre vol, vous avez le droit au remboursement des taxes prélevées par l'aéroport et les autorités publiques. Si vous souhaitez percevoir ce remboursement, renseignez le formulaire en cliquant ici https://www.tui.fr/pratique/remboursement-taxe-vol/.",
              'Si je suis insatisfait de ma location de vacances ?': 'Si vous souhaitez formuler une réclamation sur votre location de vacances, nous vous invitons à contacter directement notre partenaire TUI Villas au 01 74 18 03 76 tel:+33174180376 ou aller directement sur leur site sur la page "À propos de TUI Villas" rubrique "Question au service clientèle".',
              'Si je suis insatisfait de mon séjour, quelle procédure dois-je suivre ?': "Si vous souhaitez formuler une réclamation et obtenir une réponse, nous vous invitons à nous adresser un courrier (pas de traitement par télépho

In [0]:
sys.setrecursionlimit(10**6) # pour augmenter la limite

with open("faq.pick", 'wb') as pickFile: #wb pour write binary, car fichier pickle c'est du binaire
    pickle.dump(faq, pickFile)